In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime , timedelta ,date
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.cluster import KMeans
import scipy.optimize as sco
from numpy import *
from numpy.linalg import multi_dot
import cvxpy as cvx

## Picking Stocks

In [2]:
 #=========# Getting the tickers #=========# 
    
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = table[0]
df.to_csv('S&P500-Info.csv')
df.to_csv("S&P500-Symbols.csv", columns=['Symbol'])
a=df['Symbol'].values
list1 = a.tolist()


In [3]:
 #=========# Downloading the data #=========# 
stock_list = list1
start="2006-06-01"
end="2022-01-01"
stocks = yf.download(stock_list, start, end)
SP500= yf.download('^GSPC', start, end)

[*********************100%***********************]  504 of 504 completed

3 Failed downloads:
- CEG: Data doesn't exist for startDate = 1149134400, endDate = 1641013200
- BF.B: No data found for this date range, symbol may be delisted
- BRK.B: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed


In [4]:
 #=========# Choosing a time period to pick the stocks #=========# 
stocks_1=stocks['Close']
start_date="2006-06-01"
end_date="2011-02-01"
stocks_1=stocks_1.reset_index()
mask = (stocks_1['Date'] >= start_date) & (stocks_1['Date'] <= end_date)
stocks_1 = stocks_1.loc[mask]
stocks_1.set_index('Date', inplace=True)
stocks_1

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2006-06-01,25.057224,46.250000,37.990002,2.220357,NaN,21.920000,13.57,20.736956,28.080000,28.719999,...,72.769997,18.950001,61.110001,30.834999,NaN,18.296190,59.262135,35.810001,82.349998,NaN
2006-06-02,25.407724,45.849998,38.000000,2.202143,NaN,21.865000,13.71,20.693773,27.889999,29.000000,...,71.120003,19.010000,61.650002,31.110001,NaN,18.285406,59.427185,36.779999,82.919998,NaN
2006-06-05,24.563662,44.580002,37.419998,2.142857,NaN,21.490000,12.72,20.372309,27.469999,29.469999,...,69.110001,18.809999,60.049999,30.485001,NaN,17.965492,59.320389,36.389999,81.790001,NaN
2006-06-06,24.349070,46.279999,37.290001,2.132857,NaN,21.700001,12.74,20.261953,27.440001,29.840000,...,68.489998,18.850000,60.400002,30.184999,NaN,18.170382,60.475727,35.490002,82.110001,NaN
2006-06-07,24.248926,46.889999,37.189999,2.091429,NaN,21.620001,13.12,20.578623,27.030001,28.820000,...,69.790001,18.920000,58.820000,30.715000,NaN,18.547808,61.398060,34.950001,82.309998,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011-01-26,30.450644,10.800000,64.250000,12.280357,NaN,36.389999,10.00,22.430651,52.349998,33.520000,...,118.660004,23.950001,79.660004,35.959999,NaN,34.342201,53.669903,39.130001,23.290001,NaN
2011-01-27,30.364807,11.050000,64.150002,12.257500,NaN,36.820000,10.01,22.253124,52.360001,33.849998,...,118.870003,24.020000,79.879997,36.549999,NaN,34.212795,57.359222,39.459999,23.760000,NaN
2011-01-28,29.313305,10.160000,64.089996,12.003571,NaN,36.000000,9.81,21.826101,51.520000,32.490002,...,114.510002,23.549999,78.989998,35.750000,NaN,33.357296,56.873787,38.270000,23.430000,NaN


In [5]:
 #=========# Sharp ratio #=========# 
def sharp(data):
    u_1 = np.mean(data.pct_change()[1:])
    st =  np.std(data.pct_change()[1:])
    return u_1/st
sharp_ratio=sharp(stocks_1).dropna()
Returns_1=stocks_1.pct_change()[1:]
Returns_1=Returns_1.dropna(axis = 1).iloc[:-1,:]
sharp_ratio

A       0.018296
AAL     0.008757
AAP     0.030294
AAPL    0.070449
ABC     0.033423
          ...   
XRAY    0.016810
YUM     0.036240
ZBH     0.009497
ZBRA    0.014840
ZION    0.001667
Length: 449, dtype: float64

In [8]:
 #=========# Clustering #=========# 
clu=200
clustering_1=KMeans(n_clusters=clu)
clustering_1.fit(Returns_1.T)
clustering_2= clustering_1.predict(Returns_1.T)
results = pd.DataFrame({'Values': clustering_2,'Stocks':list(Returns_1)}).set_index('Stocks')

In [9]:
df2 = pd.concat([sharp_ratio, results], axis=1).reset_index()
df2.rename(columns={ df2.columns[1]: "SP" }, inplace = True)
df2

,index,SP,Values
0,A,0.018296,42.0
1,AAL,0.008757,29.0
2,AAP,0.030294,173.0
3,AAPL,0.070449,119.0
4,ABC,0.033423,18.0
...,...,...,...
444,XRAY,0.016810,18.0
445,YUM,0.036240,42.0
446,ZBH,0.009497,18.0
447,ZBRA,0.014840,130.0


In [15]:
 #=========# Choose stocks with high sharp ratio and less correlated #=========# 
df2.sort_values(["Values", "SP"], axis = 0, ascending = False, inplace = True,na_position ='first')
results_1=df2.drop_duplicates('Values')
Picked_stocks=results_1['index'].values
Picked_stocks

array(['NXPI', 'HAS', 'SPGI', 'IT', 'DE', 'BIO', 'PTC', 'NLOK', 'AOS',
       'PKI', 'JBHT', 'CCL', 'CTRA', 'MTB', 'BK', 'LUV', 'SLB', 'HUM',
       'CMA', 'LRCX', 'ADSK', 'CERN', 'VRSN', 'PKG', 'ATVI', 'MAR',
       'CTSH', 'AZO', 'JNPR', 'DLTR', 'WAT', 'JCI', 'TDG', 'ANSS', 'COO',
       'DD', 'SBNY', 'TPR', 'JPM', 'BIIB', 'PLD', 'NRG', 'CTXS', 'POOL',
       'BLK', 'NWL', 'LHX', 'RL', 'BWA', 'AXP', 'TSN', 'ADBE', 'GOOGL',
       'WHR', 'GLW', 'HST', 'EA', 'USB', 'TSCO', 'TRV', 'LKQ', 'DLR',
       'GL', 'AES', 'FCX', 'CME', 'NDAQ', 'WBD', 'BBWI', 'BAC', 'IBM',
       'WAB', 'MA', 'J', 'NUE', 'INTC', 'BSX', 'LEN', 'COF', 'PWR', 'FIS',
       'AAPL', 'MCO', 'SIVB', 'EXPE', 'CI', 'NTAP', 'MO', 'IPG', 'GS',
       'EQIX', 'ODFL', 'CMG', 'RJF', 'CF', 'REGN', 'MU', 'MOH', 'NVDA',
       'DRE', 'WDC', 'IP', 'DHI', 'WY', 'AMZN', 'SBAC', 'CMI', 'STT',
       'TRMB', 'PGR', 'NEM', 'DRI', 'OXY', 'KEY', 'CNC', 'TXT', 'MPWR',
       'CE', 'ILMN', 'SWKS', 'NVR', 'PARA', 'RF', 'ZION', 'MLM', 'ISRG

In [18]:
 #=========# Prices of the picked stocks #=========# 
Data=stocks['Close'][Picked_stocks]
SP500_1=SP500['Close']
Data=Data.dropna(axis = 1)


## Start the optmization

In [21]:
 #=========# Function to select the time period #=========# 
def data(df,t1,t2):
    
    # t = Horizons (days)
    # d = N pf funds
    # w = rolling period (days)
    # alpha = confidence level
    
    start_date = t1
    end_date = t2
    df=df.reset_index()
    mask = (df['Date'] >= start_date) & (df['Date'] <= end_date)
    df = df.loc[mask]
    df.set_index('Date', inplace=True)
    x1=np.array(df.pct_change()[1:].T.values)
    
   
    return x1

In [22]:
#=========# In sample sharp ratio #=========#       

def ISR1(a,data,sigma,M,i):
    var_1= a.T@sigma@a
    Uisr=a*data[0:-1,M+i-1]
    return (np.sum(Uisr,axis=0))/np.sqrt(var_1)

#=========# Out sample sharp ratio #=========#  

def OSR1(a,data,sigma,M,i):
    Uosr=a*data[0:-1,M+i]
    var_1= a.T@sigma@a
    return (np.sum(Uosr,axis=0))/np.sqrt(var_1)

#=========# Functions for the optmizationfor portfolio constraints #=========#  


def min_volatility(weights):
    port_vols = sqrt(multi_dot([weights.T, sigma , weights])) 
    return port_vols
def min_volatility2(weights):
    port_vols1 = sqrt(multi_dot([weights.T, sigma , weights])) 
    port_vols2 =0.01*np.sum(weights)
    port_vols=(port_vols1+port_vols2)**2
    return port_vols

#port returns
def Port_ret(weights): 
    weights = array(weights)
    port_rets = (weights.T @ u)
    return port_rets
def Port_ret2(weights,u,l=0): 
    weights = array(weights)
    port_rets = (weights.T @ u)- np.sum(weights)*l
    return port_rets
def Port_ret3(weights,l=0): 
    weights = array(weights)
    port_rets = (weights.T @ u) - np.sum(weights)*l
    return port_rets

#=========# Functions for the risk parity #=========#  

def backtest_risk(w):
    port_vols = sqrt(multi_dot([array(w).T, ( sigma), array(w)])) 
    return port_vols
def risk_contribution_(w):
    w = array(w)[:,newaxis]
    portfolio_vols = (backtest_risk(w))**2
    assets_risk_contribution = (np.multiply(array(w), (( sigma) @ array(w))))/portfolio_vols*100
    return assets_risk_contribution
def error(w):
    #a=list(returns_3) #10 assets
    assets_risk_contribution=risk_contribution_(w)
    x=array([100/N]*N)[:,newaxis]
    error=assets_risk_contribution-x
    total_error=sum(abs(error))
    return total_error

In [24]:

#=========++++++++++# Main function #=========++++++++++#
def out(data,c,M=120,l=0.02,g=1,k=0.0125,r=0.002):
# data
# M: window
# T: time period
# c: Method
# l: the uncertinity for methods 10,11 and 12
# g: gamma for u-mv
# k: the required return
# r = risk free
    T=np.shape(data)[1]
    ISR=np.zeros((T))
    OSR=np.zeros((T))
    turn_over=0

    
    global u, sigma, N
    
    N=np.shape(data)[0]-1
    e=np.ones(N)
    la=np.ones(N)*l 
    
    for i in range(T-M-1):
        
        #=========# Sample mean #=========#  

        u=np.array(np.mean(data[0:-1,i:M+i],axis=1))
        

        if c=='mkt': # Because only one asset (last row)
            sigma=np.std(data[-1,i:M+i]) 
        else:
            sigma=np.cov(data[0:-1,i:M+i])
            sigma_inv=linalg.inv(sigma)
            
        #=========# Caculating the weights depending on the methods #=========# 
        
        if c=='ew':   
            a=np.array(N*[1/N]) 
            
        elif c=='mkt':  
            a=1
            
        elif c=='mv': 
            a= (k-r)*((sigma_inv)@(u-r))/((u-r).T@sigma_inv@(u-r))
   
        elif c=='u-mv':  
            a= (M-N-2)/(M-1)*((sigma_inv)@(u-r))/g #g is the risk aversion
            
        elif c=='bs':  
            u1= (u.T@sigma_inv@e)/(e.T@sigma_inv@e)*e
            a1=(N+2)/(N+2+(M-N-2)*((u-u1).T@sigma_inv@(u-u1)))
            u2=a1*u1+(1-a1)*u
            a= (k-r)*((sigma_inv)@(u2-r))/((u2-r).T@sigma_inv@(u2-r))
            
        elif c=='min':  
            a= (sigma_inv@e)/(e.T@sigma_inv@e) 
            
        elif c=='mv-c':  
            w = N*[1./N]
            ef_cons = ({'type': 'eq', 'fun': lambda x: Port_ret2(x,u) - k},
                        {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
            bnds = tuple((0, 1) for x in range(N))    
            opt_ef = sco.minimize(min_volatility, w, method='SLSQP', bounds=bnds, constraints=ef_cons)
            a=opt_ef['x']
            
        elif c=='min-c':  
            w = N*[1./N]
            ef_cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
            bnds = tuple((0, 1) for x in range(N))    
            opt_ef = sco.minimize(min_volatility, w, method='SLSQP', bounds=bnds, constraints=ef_cons)
            a=opt_ef['x']   
            
        elif c=='bs-c': 
            u1= (u.T@sigma_inv@e)/(e.T@sigma_inv@e)*e
            a1=(N+2)/(N+2+(M-N-2)*((u-u1).T@sigma_inv@(u-u1)))
            u2=a1*u1+(1-a1)*u
            w = N*[1./N]
            ef_cons = ({'type': 'eq', 'fun': lambda x: Port_ret2(x,u2) - k},
                      {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
            bnds = tuple((0, 1) for x in range(N))    
            opt_ef = sco.minimize(min_volatility, w, method='SLSQP', bounds=bnds, constraints=ef_cons)
            a=opt_ef['x']

        elif c=='r-m-1': 
            w = N*[1./N]
            ef_cons = ({'type': 'eq', 'fun': lambda x: Port_ret2(x,u)-la.T@(np.abs(x)) - k})
            #ef_cons = ({'type': 'eq', 'fun': lambda x: Port_ret2(x,u)- k})
            bnds = tuple((0, 1) for x in range(N))    
            opt_ef = sco.minimize(min_volatility, w, method='SLSQP',bounds=bnds, constraints=ef_cons)
            a=opt_ef['x']
            
        elif c=='r-m-2':  
            w = N*[1./N]
            ef_cons = ({'type': 'eq', 'fun': lambda x: Port_ret2(x,u)-l*np.sqrt(x.T@sigma@x) - k})
            bnds = tuple((0, 1) for x in range(N))    
            opt_ef = sco.minimize(min_volatility, w, method='SLSQP',bounds=bnds, constraints=ef_cons)
            a=opt_ef['x']
            
        elif c=='dr-mv':  
            w = N*[1./N]
            ef_cons = ({'type': 'eq', 'fun': lambda x: Port_ret2(x,u,l)- k})
            bnds = tuple((0, 1) for x in range(N))    
            opt_ef = sco.minimize(min_volatility2, w, method='SLSQP',bounds=bnds,  constraints=ef_cons)
            a=opt_ef['x']
            
        #=========# Risk parity , used two ways to cheack that the weights are correct #=========#             
            
        elif c=='Risk_p_1':  
            w =[1]*N
            cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
            initial_wts =[1]*N
            Risk_Parity = sco.minimize(error,w, method='SLSQP',constraints=cons)
            a=Risk_Parity['x']
            
        elif c=='Risk_p_2': 
            b = np.random.randint(1,10,N)
            b = b/np.sum(b)
            y = cvx.Variable(N)
            obj = cvx.Minimize(cvx.quad_form(y, sigma))
            constraints = [np.sum([b[i]*cvx.log(y[i]) for i in range(N)]) >= 0]
            prob = cvx.Problem(obj,constraints)
            prob.solve()
            a=y.value/np.sum(y.value)            

            
         #=========# Sharp ratios  #=========# 
        
        if c=='mkt':
            #in sample sharp ratio
            Uisr=a*data[-1,M+i-1]
            ISR[i]=Uisr/sigma
            
            #out sample sharp ratio
            Oisr=a*data[-1,M+i]
            OSR[i]=Oisr/sigma
        else:
            #in sample sharp ratio
            ISR[i]=ISR1(a,data,sigma,M,i)

            #out sample sharp ratio
            OSR[i]=OSR1(a,data,sigma,M,i)
        
        
                    
    print('==================')
    print('Method:',c)
    print('ISR', np.mean(ISR))
    print('OSR',np.mean(OSR))



## Performance Test

In [35]:
# Random period   yes - 5 years
t1 = "2010-10-01"
t2 = "2015-03-01"
a=data(Data,t1,t2)
b=data(SP500_1,t1,t2)
Data_1=np.concatenate((a,b))

Distribution=Data_1
M=300
out(Distribution,'ew',M)
out(Distribution,'mkt',M)
out(Distribution,'mv',M)
out(Distribution,'bs',M)
out(Distribution,'r-m-1',M)
#out(Distribution,'dr-mv',M)
#out(Distribution,'min',M)
out(Distribution,'Risk_p_1',M)
#out(Distribution,'Risk_p_2',M)

Method: ew
ISR 0.06797819719910368
OSR 0.06895576533725153
Method: mkt
ISR 0.05616181892486291
OSR 0.0571293485864602
Method: mv
ISR 0.6777818259634074
OSR -0.22706057979969646
Method: bs
ISR 0.30137413999135687
OSR -0.2366010657035375
Method: r-m-1
ISR 0.06925512867988044
OSR 0.06553916776166638
Method: dr-mv
ISR 0.06158772022670762
OSR 0.07258903383003604
Method: Risk_p_1
ISR 0.05727154231175917
OSR 0.07267887377690788


In [36]:
# 2008 financial crisis - no
t1="2006-06-01"
t2="2010-02-01"
a=data(Data,t1,t2)
b=data(SP500_1,t1,t2)
Data_1=np.concatenate((a,b))
Distribution=Data_1
M=300
out(Distribution,'ew',M)
out(Distribution,'mkt',M)
out(Distribution,'mv',M)
out(Distribution,'bs',M)
out(Distribution,'r-m-1',M)
out(Distribution,'dr-mv',M)
#out(Distribution,'min',M)
out(Distribution,'Risk_p_1',M)
#out(Distribution,'Risk_p_2',M)


Method: ew
ISR -0.005214852370715086
OSR -0.0054378933494261955
Method: mkt
ISR -0.023398731812489738
OSR -0.024233006232780487
Method: mv
ISR 0.7968632256649995
OSR -0.07458646354398196
Method: bs
ISR 0.4646676105239703
OSR 0.09851285384079969
Method: r-m-1
ISR -0.0076783741955992095
OSR -0.0025903164968366843
Method: dr-mv
ISR -0.008418261206855541
OSR -0.005831773628880697
Method: Risk_p_1
ISR -0.008872563090430633
OSR -0.014974198132810413


In [37]:
#Covid - no
t1 = "2018-10-01"
t2 = "2021-03-30"
a=data(Data,t1,t2)
b=data(SP500_1,t1,t2)
Data_1=np.concatenate((a,b))
Distribution=Data_1
M=300
out(Distribution,'ew',M)
out(Distribution,'mkt',M)
out(Distribution,'mv',M)
out(Distribution,'bs',M)
out(Distribution,'r-m-1',M)
out(Distribution,'dr-mv',M)
#out(Distribution,'min',M)
out(Distribution,'Risk_p_1',M)
#out(Distribution,'Risk_p_2',M)

Method: ew
ISR 0.01107622530405051
OSR 0.007215680403859098
Method: mkt
ISR 0.011784268496651786
OSR 0.009384933878747719
Method: mv
ISR 0.6458497961365337
OSR -0.04282578056481789
Method: bs
ISR 0.3438710052760708
OSR 0.054822637651894095
Method: r-m-1
ISR 0.012344285870544864
OSR 0.012908851531428798
Method: dr-mv
ISR 0.015398931060500396
OSR 0.010973713204867588
Method: Risk_p_1
ISR -0.03660443939896246
OSR -0.040170318631201254


In [38]:
# 2008 financial crisis  -  yes
t1 = "2007-10-01"
t2 = "2009-03-30"
a=data(Data,t1,t2)
b=data(SP500_1,t1,t2)
Data_1=np.concatenate((a,b))
Distribution=Data_1
M=300
out(Distribution,'ew',M)
out(Distribution,'mkt',M)
out(Distribution,'mv',M)
out(Distribution,'bs',M)
out(Distribution,'r-m-1',M)
#out(Distribution,'dr-mv',M)
#out(Distribution,'min',M)
out(Distribution,'Risk_p_1',M)
#out(Distribution,'Risk_p_2',M)

Method: ew
ISR 0.007014901009034488
OSR 9.97964993093924e-05
Method: mkt
ISR -0.0035386396582815817
OSR -0.009755221250253148
Method: mv
ISR 0.26302827234328363
OSR -0.041191083771033726
Method: bs
ISR 0.1564691768227921
OSR -0.007622898608878588
Method: r-m-1
ISR 0.005156635729919181
OSR 0.0010378674434993068
Method: dr-mv
ISR 0.007066785229155836
OSR 0.005331760406873514
Method: Risk_p_1
ISR 0.007881365271217267
OSR 0.0010215462459940003


In [39]:
# The whole time period -- no
t1="2009-06-01"
t2 = "2012-12-30"
a=data(Data,t1,t2)
b=data(SP500_1,t1,t2)
Data_1=np.concatenate((a,b))
Distribution=Data_1
M=300
out(Distribution,'ew',M)
out(Distribution,'mkt',M)
out(Distribution,'mv',M)
out(Distribution,'bs',M)
out(Distribution,'r-m-1',M)
out(Distribution,'dr-mv',M)
#out(Distribution,'min',M)
out(Distribution,'Risk_p_1',M)
#out(Distribution,'Risk_p_2',M)

Method: ew
ISR 0.028734777146839526
OSR 0.027529134766674374
Method: mkt
ISR 0.02193270746780605
OSR 0.020686029313793975
Method: mv
ISR 0.6872287185456927
OSR -0.10099247336191901
Method: bs
ISR 0.32413377916649205
OSR -0.15666654650062536
Method: r-m-1
ISR 0.024994701829830206
OSR 0.030933306355198163
Method: dr-mv
ISR 0.024656194756877216
OSR 0.029676619010569984
Method: Risk_p_1
ISR 0.03185567220333533
OSR 0.030286211939185428


In [40]:
#Covid  -  no
t1 = "2018-10-01"
t2 = "2020-03-30"
a=data(Data,t1,t2)
b=data(SP500_1,t1,t2)
Data_1=np.concatenate((a,b))
Distribution=Data_1
M=300
out(Distribution,'ew',M)
out(Distribution,'mkt',M)
out(Distribution,'mv',M)
out(Distribution,'bs',M)
out(Distribution,'r-m-1',M)
out(Distribution,'dr-mv',M)
#out(Distribution,'min',M)
out(Distribution,'Risk_p_1',M)
#out(Distribution,'Risk_p_2',M)

Method: ew
ISR -0.05536002495152932
OSR -0.06660962367298245
Method: mkt
ISR -0.04017899915527321
OSR -0.04931114318792654
Method: mv
ISR 0.27849280060346066
OSR 0.3198948692434235
Method: bs
ISR 0.1526544739890221
OSR 0.41034777113306004
Method: r-m-1
ISR -0.05493267801031292
OSR -0.0626880367940869
Method: dr-mv
ISR -0.05403511707469523
OSR -0.06440265453901906
Method: Risk_p_1
ISR -0.058419171035528614
OSR -0.07111755329171154


In [41]:
#Covid - no
t1="2018-01-01"
t2 = "2021-01-01"
a=data(Data,t1,t2)
b=data(SP500_1,t1,t2)
Data_1=np.concatenate((a,b))
Distribution=Data_1
M=300
out(Distribution,'ew',M)
out(Distribution,'mkt',M)
out(Distribution,'mv',M)
out(Distribution,'bs',M)
out(Distribution,'r-m-1',M)
out(Distribution,'dr-mv',M)
#out(Distribution,'min',M)
out(Distribution,'Risk_p_1',M)
#out(Distribution,'Risk_p_2',M)

Method: ew
ISR 0.013410444521800657
OSR 0.010989686603286053
Method: mkt
ISR 0.020684604111236193
OSR 0.01858198198263521
Method: mv
ISR 0.7186632647846862
OSR 0.0029533243830411606
Method: bs
ISR 0.37723587827674526
OSR 0.074423730481744


KeyboardInterrupt: 

In [67]:
#Covid-- run this - no didnt even beat the markt
t1="2006-06-01"
t2 = "2021-12-30"
a=data(Data,t1,t2)
b=data(SP500_1,t1,t2)
Data_1=np.concatenate((a,b))
Distribution=Data_1
M=300
out(Distribution,'ew',M)
out(Distribution,'mkt',M)
out(Distribution,'mv',M)
#out(Distribution,'min',M)
out(Distribution,'r-m-1',M)
out(Distribution,'Risk_p_1',M)
#out(Distribution,'Risk_p_2',M)

Method: ew
ISR 0.04262407753771986
OSR 0.04146019936895296
Method: mkt
ISR 0.03400362973293275
OSR 0.03267404979326579
Method: mv
ISR 1.0032847931837645
OSR -0.1076873590738662
Method: r-m-1
ISR 0.0401751148759758
OSR 0.042859175823474795
Method: Risk_p_1
ISR 0.02010200166688147
OSR 0.02939517025353841


In [63]:
#Covid--yes - remove the min
t1="2019-09-01"
t2 = "2020-10-01"
a=data(Data,t1,t2)
b=data(SP500_1,t1,t2)
Data_1=np.concatenate((a,b))
Distribution=Data_1
M=250
out(Distribution,'ew',M)
out(Distribution,'mkt',M)
out(Distribution,'mv',M)
#out(Distribution,'min',M)
out(Distribution,'r-m-1',M)
out(Distribution,'Risk_p_1',M)


Method: ew
ISR -0.004914456947065316
OSR -0.005293786848606275
Method: mkt
ISR -0.007131728320878827
OSR -0.00694643529923726
Method: mv
ISR 0.1159449384693909
OSR -0.046126475134093854
Method: min
ISR -0.02539971635641915
OSR -0.1302406921787372
Method: r-m-1
ISR -0.004970104404205128
OSR -0.004603394828920157
Method: Risk_p_1
ISR -0.005314200675019327
OSR -0.0020862345009654906


In [62]:
#Covid - no
t1="2019-10-01"
t2 = "2020-12-01"
a=data(Data,t1,t2)
b=data(SP500_1,t1,t2)
Data_1=np.concatenate((a,b))
Distribution=Data_1
M=250
out(Distribution,'ew',M)
out(Distribution,'mkt',M)
out(Distribution,'mv',M)
out(Distribution,'min',M)
out(Distribution,'Risk_p_1',M)

Method: ew
ISR 0.02256823548464942
OSR 0.018368464927243488
Method: mkt
ISR 0.015913153950979864
OSR 0.012621530309230891
Method: mv
ISR 0.22623350616554927
OSR -0.08703230110010383
Method: min
ISR -0.015222489156388444
OSR -0.09903485707258443
Method: Risk_p_1
ISR -0.024276118925899263
OSR -0.020917014054924514


In [64]:
# 2008 financial crisis  -  yes
t1 = "2007-10-01"
t2 = "2009-03-30"
a=data(Data,t1,t2)
b=data(SP500_1,t1,t2)
Data_1=np.concatenate((a,b))
Distribution=Data_1
M=300

out(Distribution,'min',M)


Method: min
ISR -0.009822367762351056
OSR -0.03287634815207876


In [72]:
# Random period -- 10 years
t1 = "2010-10-01"
t2 = "2020-10-01"
a=data(Data,t1,t2)
b=data(SP500_1,t1,t2)
Data_1=np.concatenate((a,b))

Distribution=Data_1
M=300

M=300
out(Distribution,'ew',M)
out(Distribution,'mkt',M)
out(Distribution,'mv',M)
#out(Distribution,'min',M)
#out(Distribution,'r-m-1',M)
#out(Distribution,'Risk_p_1',M)
out(Distribution,'Risk_p_2',M)


Method: ew
ISR 0.05104598367603114
OSR 0.04998429105400353
Method: mkt
ISR 0.046056248376473645
OSR 0.04503762838038599
Method: mv
ISR 0.9362656176491607
OSR -0.13777566911017522


C:\Users\Admin\anaconda3\lib\site-packages\cvxpy\problems\problem.py:1278: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


Method: Risk_p_2
ISR 0.055483100027741025
OSR 0.052876078412688286


In [73]:
# Random period - 8 years
t1 = "2010-10-01"
t2 = "2018-10-01"
a=data(Data,t1,t2)
b=data(SP500_1,t1,t2)
Data_1=np.concatenate((a,b))

Distribution=Data_1
M=300

M=300
out(Distribution,'ew',M)
out(Distribution,'mkt',M)
out(Distribution,'mv',M)
#out(Distribution,'min',M)
#out(Distribution,'r-m-1',M)
#out(Distribution,'Risk_p_1',M)
out(Distribution,'Risk_p_2',M)


Method: ew
ISR 0.06591778545680958
OSR 0.06565910341312883
Method: mkt
ISR 0.05573908330623296
OSR 0.05519559949733385
Method: mv
ISR 0.8788433210652111
OSR -0.16732406796813423


C:\Users\Admin\anaconda3\lib\site-packages\cvxpy\problems\problem.py:1278: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


Method: Risk_p_2
ISR 0.06933174319413282
OSR 0.06747338539471213


In [74]:
#i will do for 6 7 8 and 

In [77]:
# Random period - 8 years
t1 = "2010-10-01"
t2 = "2018-10-01"
a=data(Data,t1,t2)
b=data(SP500_1,t1,t2)
Data_1=np.concatenate((a,b))

Distribution=Data_1
M=300

M=300
#out(Distribution,'ew',M)
#out(Distribution,'mkt',M)
#out(Distribution,'mv',M)
#out(Distribution,'min',M)
out(Distribution,'r-m-1',M)
#out(Distribution,'Risk_p_1',M)
#out(Distribution,'Risk_p_2',M)


Method: r-m-1
ISR 0.06684393772687522
OSR 0.065391871956576


In [ ]:
## run this tmorow

In [78]:
# Random period -- 10 years
t1 = "2010-10-01"
t2 = "2020-10-01"
a=data(Data,t1,t2)
b=data(SP500_1,t1,t2)
Data_1=np.concatenate((a,b))

Distribution=Data_1
M=300

M=300
out(Distribution,'ew',M)
out(Distribution,'mkt',M)
out(Distribution,'mv',M)
#out(Distribution,'min',M)
#out(Distribution,'r-m-1',M)
out(Distribution,'Risk_p_1',M)
#out(Distribution,'Risk_p_2',M)

Method: ew
ISR 0.05104598367603114
OSR 0.04998429105400353
Method: mkt
ISR 0.046056248376473645
OSR 0.04503762838038599
Method: mv
ISR 0.9362656176491607
OSR -0.13777566911017522
Method: Risk_p_1
ISR 0.02838429244307624
OSR 0.046160309076927704


In [79]:
# Random period -- 10 years
t1 = "2010-10-01"
t2 = "2020-10-01"
a=data(Data,t1,t2)
b=data(SP500_1,t1,t2)
Data_1=np.concatenate((a,b))

Distribution=Data_1
M=300

M=300
out(Distribution,'ew',M)
out(Distribution,'mkt',M)
out(Distribution,'mv',M)
#out(Distribution,'min',M)
out(Distribution,'r-m-1',M)
#out(Distribution,'Risk_p_1',M)
#out(Distribution,'Risk_p_2',M)

Method: ew
ISR 0.05104598367603114
OSR 0.04998429105400353
Method: mkt
ISR 0.046056248376473645
OSR 0.04503762838038599
Method: mv
ISR 0.9362656176491607
OSR -0.13777566911017522
Method: r-m-1
ISR 0.049613743702995705
OSR 0.049777627457166865


In [80]:
# Random period -- 8 years
t1 = "2010-10-01"
t2 = "2018-10-01"
a=data(Data,t1,t2)
b=data(SP500_1,t1,t2)
Data_1=np.concatenate((a,b))

Distribution=Data_1
M=300

M=300
out(Distribution,'ew',M)
out(Distribution,'mkt',M)
out(Distribution,'mv',M)
#out(Distribution,'min',M)
#out(Distribution,'r-m-1',M)
out(Distribution,'Risk_p_1',M)
#out(Distribution,'Risk_p_2',M)

Method: ew
ISR 0.06591778545680958
OSR 0.06565910341312883
Method: mkt
ISR 0.05573908330623296
OSR 0.05519559949733385
Method: mv
ISR 0.8788433210652111
OSR -0.16732406796813423
Method: Risk_p_1
ISR 0.03706005523319284
OSR 0.060554655105770364


In [ ]:
#add three 

In [81]:
# Random period -- 3 years
t1 = "2010-10-01"
t2 = "2013-10-01"
a=data(Data,t1,t2)
b=data(SP500_1,t1,t2)
Data_1=np.concatenate((a,b))

Distribution=Data_1
M=300

M=300
out(Distribution,'ew',M)
out(Distribution,'mkt',M)
out(Distribution,'mv',M)
#out(Distribution,'min',M)
out(Distribution,'r-m-1',M)
out(Distribution,'Risk_p_1',M)
#out(Distribution,'Risk_p_2',M)

Method: ew
ISR 0.050930281497627704
OSR 0.05258143420221875
Method: mkt
ISR 0.04042886462093606
OSR 0.04121672110011139
Method: mv
ISR 0.548119682588515
OSR -0.22405539508462163
Method: r-m-1
ISR 0.05198909064000982
OSR 0.0504884724059047
Method: Risk_p_1
ISR 0.05207593706305296
OSR 0.053352897490806135


In [ ]:
# save the stocks names

In [82]:
Picked_stocks

array(['NXPI', 'HAS', 'SPGI', 'IT', 'DE', 'BIO', 'PTC', 'NLOK', 'AOS',
       'PKI', 'JBHT', 'CCL', 'CTRA', 'MTB', 'BK', 'LUV', 'SLB', 'HUM',
       'CMA', 'LRCX', 'ADSK', 'CERN', 'VRSN', 'PKG', 'ATVI', 'MAR',
       'CTSH', 'AZO', 'JNPR', 'DLTR', 'WAT', 'JCI', 'TDG', 'ANSS', 'COO',
       'DD', 'SBNY', 'TPR', 'JPM', 'BIIB', 'PLD', 'NRG', 'CTXS', 'POOL',
       'BLK', 'NWL', 'LHX', 'RL', 'BWA', 'AXP', 'TSN', 'ADBE', 'GOOGL',
       'WHR', 'GLW', 'HST', 'EA', 'USB', 'TSCO', 'TRV', 'LKQ', 'DLR',
       'GL', 'AES', 'FCX', 'CME', 'NDAQ', 'WBD', 'BBWI', 'BAC', 'IBM',
       'WAB', 'MA', 'J', 'NUE', 'INTC', 'BSX', 'LEN', 'COF', 'PWR', 'FIS',
       'AAPL', 'MCO', 'SIVB', 'EXPE', 'CI', 'NTAP', 'MO', 'IPG', 'GS',
       'EQIX', 'ODFL', 'CMG', 'RJF', 'CF', 'REGN', 'MU', 'MOH', 'NVDA',
       'DRE', 'WDC', 'IP', 'DHI', 'WY', 'AMZN', 'SBAC', 'CMI', 'STT',
       'TRMB', 'PGR', 'NEM', 'DRI', 'OXY', 'KEY', 'CNC', 'TXT', 'MPWR',
       'CE', 'ILMN', 'SWKS', 'NVR', 'PARA', 'RF', 'ZION', 'MLM', 'ISRG

In [83]:
df10 = pd.DataFrame (Picked_stocks, columns = ['column_name'])

In [84]:
df10.to_csv('stocks.csv')